## load packages

In [2]:
library(dplyr)
library(lubridate)
library(tableone)
library(DescTools)
library(biostat3)
library(survminer)
library(mice)
library(foreach)
library(mgcv)
library(doParallel)
library(mitools)
library(survey)
library(rms)
library(tidyr)
library(data.table)

In [3]:
## HR display function
OR.result.tidy <- function(result) {
    res <- result$coefficients[2, c(1, 2)]
    OR <- exp(res[1])
    lci <- exp(res[1] - qnorm(0.975) * res[2])
    uci <- exp(res[1] + qnorm(0.975) * res[2])
    res <- data.frame(OR, lci, uci)
    colnames(res) <- c('OR', 'lci', 'uci')
    res <- res %>% 
        data.frame() %>% 
        mutate_at(vars(OR, lci, uci), funs(round(., 2))) 
    res <- res %>% 
        mutate(OR = paste0(res[, 1], " (", res[, 2], "-", res[, 3], ")")) %>% 
        cbind(group = c('discontinuation')) %>% 
        dplyr::select(c(4, 1))
    res <- rbind(data.frame(group = 'reinitiation', OR = 'Ref'), res) 
    res$group <- as.character(res$group)
    return(res)
}

## import data

In [4]:
load('dta_tidied.R')

In [5]:
dta_clone_death <- dta_clone %>% 
    arrange(X, lopnr, index_date) %>% 
    group_by(X, lopnr) %>% 
    mutate(death_filter = cumsum(outcome_death), 
           censor_filter = cumsum(censor)) %>% 
    filter(death_filter <= 1 & censor_filter <= 1) %>% 
    ungroup() %>% 
    dplyr::select(c('lopnr', 'ID', 'X', 
                    'index_date', 'after_index_date', 'rank', 
                    'age', 'female', 'index_year', 'duration', 
                    'potassium_value', 'potassium_cat', 
                    'primary_care_num', 'outpatient_num', 'inpatient_num', 
                    'cov_diabetes', 'cov_hypertension', 
                    'cov_MI', 'cov_CHF', 'cov_cerebrovascular_disease', 'cov_PVD', 
                    'cov_COPD',  
                    'cov_beta_blocker', 'cov_CCB', 'cov_diuretic', 'cov_MRA', 'cov_SPS', 
                    'cov_statin', 
                    'cov_antiplatelet_agent', 
                    'cov_egfr', 'cov_egfr_cat', 'cov_acr', 'cov_acr_cat', 
                    'cov_potassium', 'cov_hospitalization', 
                    'outcome_death', 'censor', 'treatment'))

In [6]:
dta_clone_death_imp1 <- dta_clone_death %>% group_by(ID, lopnr) %>% 
    arrange(ID, lopnr, index_date) %>% 
    mutate(cov_diabetes_lag = lag(cov_diabetes, 1), 
           cov_hypertension_lag = lag(cov_hypertension, 1), 
           cov_MI_lag = lag(cov_MI, 1), 
           cov_CHF_lag = lag(cov_CHF, 1), 
           cov_cerebrovascular_disease_lag = lag(cov_cerebrovascular_disease, 1), 
           cov_PVD_lag = lag(cov_PVD, 1), 
           cov_COPD_lag = lag(cov_COPD, 1), 
           cov_beta_blocker_lag = lag(cov_beta_blocker, 1), 
           cov_CCB_lag = lag(cov_CCB, 1), 
           cov_diuretic_lag = lag(cov_diuretic, 1), 
           cov_MRA_lag = lag(cov_MRA, 1), 
           cov_SPS_lag = lag(cov_SPS, 1), 
           cov_statin_lag = lag(cov_statin, 1), 
           cov_antiplatelet_agent_lag = lag(cov_antiplatelet_agent, 1), 
           cov_egfr_lag = lag(cov_egfr, 1), 
           cov_egfr_cat_lag = lag(cov_egfr_cat, 1), 
           cov_acr_lag = lag(cov_acr, 1), 
           cov_acr_cat_lag = lag(cov_acr_cat, 1), 
           cov_potassium_lag = lag(cov_potassium, 1), 
           cov_hospitalization_lag = lag(cov_hospitalization, 1)) %>% ungroup()

## model conditional probability

In [7]:
x_adjusted_var <- c('age', 'female', 'index_year', 'duration', 'potassium_value', 
                    'primary_care_num', 'outpatient_num', 'inpatient_num', 
                    'cov_diabetes', 'cov_hypertension', 
                    'cov_MI', 'cov_CHF', 'cov_cerebrovascular_disease', 'cov_PVD', 
                    'cov_COPD',  
                    'cov_beta_blocker', 'cov_CCB', 'cov_diuretic', 'cov_MRA', 'cov_SPS', 
                    'cov_statin', 
                    'cov_antiplatelet_agent', 
                    'cov_egfr', 
                    # 'rcs(cov_potassium, 12)',
                    'cov_hospitalization')

In [8]:
x_adjusted_var_lag <- c('age', 'female', 'index_year', 'duration', 'potassium_value', 
                        'primary_care_num', 'outpatient_num', 'inpatient_num', 
                        'cov_diabetes_lag', 'cov_hypertension_lag', 
                        'cov_MI_lag', 'cov_CHF_lag', 'cov_cerebrovascular_disease_lag', 'cov_PVD_lag', 
                        'cov_COPD_lag',  
                        'cov_beta_blocker_lag', 'cov_CCB_lag', 'cov_diuretic_lag', 'cov_MRA_lag', 'cov_SPS_lag', 
                        'cov_statin_lag', 
                        'cov_antiplatelet_agent_lag', 
                        'cov_egfr_lag', 
                        # 'rcs(cov_potassium_lag, 12)', 
                        'cov_hospitalization_lag')

In [9]:
dta_clone_death_imp1_discontinuation <- dta_clone_death_imp1 %>% 
    filter(X == 'discontinuation') %>% 
    arrange(ID, index_date)

In [10]:
print(paste(attr(rcspline.eval(dta_clone_death_imp1_discontinuation$rank, nk = 3, inclx = T), "knots"), collapse = ', '))

[1] "1, 7, 32"


In [11]:
## outcome conditional model
outcome_regression_formula_discontinuation <- as.formula(paste0('outcome_death ~ rcspline.eval(rank, knots = c(1, 7, 32), inclx = T) + ', 
                                                                paste(x_adjusted_var, collapse = " + "), 
                                                                sep = ' '))
outcome_model_discontinuation <- glm(outcome_regression_formula_discontinuation, 
                                     data = dta_clone_death_imp1_discontinuation, 
                                     family  = binomial(link = "logit"))

In [12]:
print(paste(attr(rcspline.eval(dta_clone_death_imp1_discontinuation %>% 
                               filter(rank %in% 1 : 6) %>% 
                               dplyr::select(rank), nk = 3, inclx = T), "knots"), collapse = ', '))

[1] "1, 2, 5"


In [13]:
## censoring conditional model
censor_regression_formula_discontinuation <- as.formula(paste0('censor ~ rcspline.eval(rank, knots = c(1, 2, 5), inclx = T) + ', 
                                                               paste(x_adjusted_var, collapse = " + "), 
                                                               sep = ' '))
censor_model_discontinuation <- glm(censor_regression_formula_discontinuation, 
                                    data = dta_clone_death_imp1_discontinuation %>% filter(rank %in% 1 : 6), 
                                    family  = binomial(link = "logit"))

In [14]:
print(paste(attr(rcspline.eval(dta_clone_death_imp1_discontinuation %>% 
                               filter(rank >= 2) %>% 
                               dplyr::select(rank), nk = 3, inclx = T), "knots"), collapse = ', '))

[1] "2, 11, 34"


In [15]:
## confounder conditional model
confounder_conditional_model <- function(confounder = 'cov_diabetes', data = dta_clone_death_imp1_discontinuation) {
    if (unique(data$X) == 'discontinuation') {
        confounder_regression_formula <- as.formula(paste0(confounder, '~ rcspline.eval(rank, knots = c(2, 11, 34), inclx = T) + ', 
                                                       paste(x_adjusted_var_lag, collapse = " + "), 
                                                       collapse = ' '))    
    } else {
            confounder_regression_formula <- as.formula(paste0(confounder, '~ rcspline.eval(rank, knots = c(4, 15, 36), inclx = T) + ', 
                                                       paste(x_adjusted_var_lag, collapse = " + "), 
                                                       collapse = ' '))
    }
    if (confounder %in% c('cov_egfr')) {
        confounder_model <- lm(confounder_regression_formula, 
                               data = data %>% filter(rank >= 2))
    } else {
        confounder_model <- glm(confounder_regression_formula, 
                                data = data %>% filter(rank >= 2), 
                                family  = binomial(link = "logit"))
    }
    return(confounder_model)
}

In [16]:
xvar_time_varying <- c('cov_diabetes', 'cov_hypertension', 
                       'cov_MI', 'cov_CHF', 'cov_cerebrovascular_disease', 'cov_PVD', 
                       'cov_COPD',  
                       'cov_beta_blocker', 'cov_CCB', 'cov_diuretic', 'cov_MRA', 'cov_SPS', 
                       'cov_statin', 'cov_antiplatelet_agent', 
                       'cov_egfr', 
                       # 'rcs(cov_potassium, 12)', 
                       'cov_hospitalization')

In [17]:
confouder_model_discontinuation <- mcMap(function(i) {
    confounder_conditional_model(confounder = i)
}, xvar_time_varying)

In [18]:
for (i in xvar_time_varying) {
    assign(paste(i, 'model_discontinuation', sep = '_'), confouder_model_discontinuation[[i]])
}

In [19]:
dta_clone_death_imp1_reinitiation <- dta_clone_death_imp1 %>% 
    filter(X == 'reinitiation') %>% 
    arrange(ID, index_date)

In [20]:
print(paste(attr(rcspline.eval(dta_clone_death_imp1_reinitiation$rank, nk = 3, inclx = T), "knots"), collapse = ', '))

[1] "3, 14, 36"


In [21]:
## outcome conditional model
outcome_regression_formula_reinitiation <- as.formula(paste0('outcome_death ~ rcspline.eval(rank, knots = c(3, 14, 36), inclx = T) + ', 
                                                             paste(x_adjusted_var, collapse = " + "), 
                                                             sep = ' '))
outcome_model_reinitiation <- glm(outcome_regression_formula_reinitiation, 
                                  data = dta_clone_death_imp1_reinitiation, 
                                  family  = binomial(link = "logit"))

In [22]:
## censoring conditional model
censor_regression_formula_reinitiation <- as.formula(paste0('censor ~ ', 
                                                            paste(x_adjusted_var, collapse = " + "), 
                                                            sep = ' '))
censor_model_reinitiation <- glm(censor_regression_formula_reinitiation, 
                                 data = dta_clone_death_imp1_reinitiation %>% filter(rank %in% c(6)), 
                                 family  = binomial(link = "logit"))

In [23]:
print(paste(attr(rcspline.eval(dta_clone_death_imp1_reinitiation %>% 
                               filter(rank >= 2) %>% 
                               dplyr::select(rank), nk = 3, inclx = T), "knots"), collapse = ', '))

[1] "4, 15, 36"


In [24]:
confouder_model_reinitiation <- mcMap(function(i) {
    confounder_conditional_model(confounder = i, data = dta_clone_death_imp1_reinitiation)
}, xvar_time_varying)

In [25]:
for (i in xvar_time_varying) {
    assign(paste(i, 'model_reinitiation', sep = '_'), confouder_model_reinitiation[[i]])
}

In [26]:
# save(list = ls(), file = 'before MC all.R')

## Generating time-varying confounders and outcome in Monte Carlo sample 

In [27]:
# load('before MC all.R')

In [28]:
MC_data_preparation <- function(data = dta_discontinuation_MC, 
                                ID_sample = ID_random_sample_discontinuation, times = 1, 
                                n_month = 60) {
    sub_data <- function(i, j = 1) {
        dta <- data %>% filter(ID == i) %>% mutate(ID = j) %>% data.frame()
        return(dta)
    }
    data <- rbindlist(mapply(sub_data, ID_sample,  1 : length(ID_sample), SIMPLIFY = F))
    data <- data %>% 
        dplyr::select(c('lopnr', 'ID', 'X', 'rank', x_adjusted_var)) %>% 
        mutate(ID = paste('ID', ID, sep = '_'), 
               outcome_death = NA)
    for (k in paste(xvar_time_varying, 'lag', sep = '_')) {
        data[[k]] <- NA
    }
    n_month = n_month 
    data <- data %>% 
        slice(rep(row_number(), n_month)) %>% 
        arrange(ID) %>% 
        group_by(ID) %>% 
        mutate(rank = cumsum(rank)) %>% 
        ungroup()
    for (m in xvar_time_varying) {
        data[[m]] <- ifelse(data$rank == 1, data[[m]], NA)
    }
    data <- data %>% slice(rep(row_number(), times))
    return(data)
}

In [29]:
dta_discontinuation_MC <- dta_clone_death_imp1_discontinuation %>% filter(rank == 1)
ID_random_sample_discontinuation <- 1 : length(unique(dta_clone_death_imp1_discontinuation$ID))
dta_discontinuation_MC = MC_data_preparation()
dta_reinitiation_MC <- dta_clone_death_imp1_reinitiation %>% filter(rank == 1)
ID_random_sample_reinitiation <- ID_random_sample_discontinuation + length(ID_random_sample_discontinuation)
dta_reinitiation_MC = MC_data_preparation(data = dta_reinitiation_MC, 
                                          ID_sample = ID_random_sample_reinitiation)

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(x_adjusted_var)` instead of `x_adjusted_var` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [30]:
MC_data_generation_per <- function(i = 2, 
                                   type = 'discontinuation', 
                                   environment = globalenv()) {
    data = get(paste(c('dta', type, 'MC', i - 1), collapse = '_'), envir = environment) 
    ## Note from global environment
    ## print(head(data)) 
    alldata = get(paste(c('dta', type, 'MC'), collapse = '_'), envir = environment) 
    ## Note from global environment
    ## print(head(alldata)) 
    dta_MC_per_generated <- alldata %>% filter(rank == i) %>% 
        dplyr::select(c('lopnr', 'ID', 'X', 'rank', 'age', 'female', 'index_year', 'duration', 
                        'potassium_value', 
                        'primary_care_num', 'outpatient_num', 'inpatient_num', 
                        xvar_time_varying, 'outcome_death')) %>% 
        inner_join(data %>% filter(outcome_death == 0) %>% 
                       dplyr::select(c('ID', xvar_time_varying)), 
                   by = 'ID', suffix = c('', '_lag'))
    ## add values for time-varying confounders
    for (j in xvar_time_varying) {
        model <- get(paste(c(j, 'model', type), collapse = '_'), envir = .GlobalEnv) 
        ##Note from global environment
        ## print(model)
        if (j %in% c('cov_egfr')) {
            mean_cov = predict(model, newdata = dta_MC_per_generated)
            set.seed(060305)
            dta_MC_per_generated[[j]] = rnorm(length(mean_cov), 
                                              mean = mean_cov, 
                                              sd = sd(get(paste(c('dta_clone_death_imp1_', type), 
                                                                collapse = ''))[[j]]))
        } else {
            p_cov = predict(model, newdata = dta_MC_per_generated, type = "response")
            set.seed(060305)
            dta_MC_per_generated[[j]] = rbinom(length(p_cov), 1, p_cov)
        }

    }
    ## add value for outcome
    model <- get(paste(c('outcome', 'model', type), collapse = '_'), envir = .GlobalEnv) 
    ##Note from global environment    
    ## print(model)
    p_outcome = predict(model, newdata = dta_MC_per_generated, type = "response")
    set.seed(060305)
    dta_MC_per_generated$outcome_death = rbinom(length(p_outcome), 1, p_outcome)
    return(dta_MC_per_generated)
}

In [31]:
dta_discontinuation_MC_1 <- dta_discontinuation_MC %>% filter(rank == 1)
p_outcome = predict(outcome_model_discontinuation, newdata = dta_discontinuation_MC_1, type = "response")
set.seed(060305)
dta_discontinuation_MC_1$outcome_death = rbinom(length(p_outcome), 1, p_outcome)

In [32]:
for (i in 2 : 60) {
    dta_discontinuation_MC_per <- MC_data_generation_per(i = i, type = 'discontinuation')
    assign(paste('dta_discontinuation_MC_', i, sep = ''), 
            dta_discontinuation_MC_per)
}
dta_discontinuation_MC <- foreach (i = 1 : 60, .combine = rbind) %do% {
    data <- get(paste('dta_discontinuation_MC_', i, sep = ''))
}

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(xvar_time_varying)` instead of `xvar_time_varying` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [33]:
dta_reinitiation_MC_1 <- dta_reinitiation_MC %>% filter(rank == 1)
p_outcome = predict(outcome_model_reinitiation, newdata = dta_reinitiation_MC_1, type = "response")
set.seed(060305)
outcome = rbinom(length(p_outcome), 1, p_outcome)
dta_reinitiation_MC_1$outcome_death = outcome
for (i in 2 : 60) {
    data = get(paste('dta_reinitiation_MC_', i - 1, sep = ''))
    dta_reinitiation_MC_per <- MC_data_generation_per(i = i, type = 'reinitiation')
    assign(paste('dta_reinitiation_MC_', i, sep = ''), dta_reinitiation_MC_per)
}
dta_reinitiation_MC <- foreach (i = 1 : 60, .combine = rbind) %do% {
    data <- get(paste('dta_reinitiation_MC_', i, sep = ''))
}

In [34]:
dta_MC <- rbind(dta_discontinuation_MC, dta_reinitiation_MC) %>% mutate(X = ifelse(X == 'reinitiation', 0, 1))

## causal effect estimation

In [35]:
causal_effect_estimation <- function(data = dta_MC, month = 36) {
    regression_formula <- as.formula('outcome_death ~ X + rms::rcs(rank, 3)')
    death.logistic.adjusted <- glm(regression_formula, data = data, family = binomial(link = "logit"))
    death.res.adjusted <- summary(death.logistic.adjusted)
    death_risk_summary_discontinuation <- data %>% filter(X == 1) %>% 
        ungroup() %>% group_by(rank) %>% 
        summarise(p_surv = 1 - sum(outcome_death) / length(outcome_death)) %>% 
        mutate(p_cumsurv = cumprod(p_surv), 
               p_cumdeath = 1 - p_cumsurv) 
    death_risk_summary_reinitiation <- data %>% filter(X == 0) %>% 
        ungroup() %>% group_by(rank) %>% 
        summarise(p_surv = 1 - sum(outcome_death) / length(outcome_death)) %>% 
        mutate(p_cumsurv = cumprod(p_surv), 
               p_cumdeath = 1 - p_cumsurv) 
    ##========================================================================================================
    risk_discontinuation_3_y <- death_risk_summary_discontinuation %>% 
        filter(rank == month) %>% 
        dplyr::select(p_cumdeath) %>% as.numeric()
    risk_reinitiation_3_y <- death_risk_summary_reinitiation %>% 
        filter(rank == month) %>% 
        dplyr::select(p_cumdeath) %>% as.numeric()
    RD <- (risk_discontinuation_3_y - risk_reinitiation_3_y) * 100
    RR <- (risk_discontinuation_3_y / risk_reinitiation_3_y)
    risk_res <- round(c(risk_discontinuation_3_y * 100, risk_reinitiation_3_y * 100, RD, RR, 
                        exp(death.res.adjusted$coefficients[2, 1])), 2)
    names(risk_res) <- c('3 year risk for discontinuation (%)', 
                         '3 year risk for reinitiation (%)', 
                         'risk difference (%)', 
                         'risk ratio', 
                         'hazard ratio')
    risk_res <- risk_res %>% t()
    return(risk_res)
}

In [36]:
main_point_estimation <- causal_effect_estimation()
main_point_estimation

`summarise()` ungrouping output (override with `.groups` argument)

`summarise()` ungrouping output (override with `.groups` argument)



3 year risk for discontinuation (%),3 year risk for reinitiation (%),risk difference (%),risk ratio,hazard ratio
62.25,56.8,5.44,1.1,1.19


In [37]:
## bootstrap on 500 sample
no_of_sample = 250
cl <- parallel::makeCluster(3, type = "SOCK")
registerDoParallel(cl)

In [38]:
BS_data_generation_per <- function(i = 2, 
                                   type = 'discontinuation', 
                                   environment = globalenv()) {
    data = get(paste(c('dta', type, 'BS', i - 1), collapse = '_')) 
    alldata = get(paste(c('dta', type, 'BS'), collapse = '_')) 
    dta_BS_per_generated <- alldata %>% filter(rank == i) %>% 
                dplyr::select(c('lopnr', 'ID', 'X', 'rank', 'age', 'female', 'index_year', 'duration', 
                        'potassium_value', 
                        'primary_care_num', 'outpatient_num', 'inpatient_num', 
                        xvar_time_varying, 'outcome_death')) %>% 
        inner_join(data %>% filter(outcome_death == 0) %>% 
                       dplyr::select(c('ID', xvar_time_varying)), 
                   by = 'ID', suffix = c('', '_lag'))
    ## add values for time-varying confounders
    for (j in xvar_time_varying) {
        model <- get(paste(c(j, 'model', type), collapse = '_')) 
        if (j %in% c('cov_egfr')) {
            mean_cov = predict(model, newdata = dta_BS_per_generated)
            set.seed(060305)
            dta_BS_per_generated[[j]] = rnorm(length(mean_cov), 
                                              mean = mean_cov, 
                                              sd = sd(get(paste(c('dta_clone_death_imp1_', type), 
                                                                collapse = ''))[[j]]))
        } else {
            p_cov = predict(model, newdata = dta_BS_per_generated, type = "response")
            set.seed(060305)
            dta_BS_per_generated[[j]] = rbinom(length(p_cov), 1, p_cov)
        }

    }
    ## add value for outcome
    model <- get(paste(c('outcome', 'model', type), collapse = '_')) 
    p_outcome = predict(model, newdata = dta_BS_per_generated, type = "response")
    set.seed(060305)
    dta_BS_per_generated$outcome_death = rbinom(length(p_outcome), 1, p_outcome)
    return(dta_BS_per_generated)
}

In [39]:
CI_res <- foreach (j = 1 : no_of_sample, .combine = rbind, 
                   .export = ls(globalenv()), 
                   .packages = c("foreach", "dplyr", 'data.table', 'Hmisc')) %dopar% {
    dta_discontinuation_BS <- dta_clone_death_imp1_discontinuation %>% filter(rank == 1)
    set.seed(j)
    ID_BS_discontinuation <- sample(unique(dta_clone_death_imp1_discontinuation$ID), 
                                    size = length(unique(dta_clone_death_imp1_discontinuation$ID)), 
                                    replace = T) 
    dta_discontinuation_BS = MC_data_preparation(data = dta_discontinuation_BS, 
                                                 ID_sample = ID_BS_discontinuation)
    dta_reinitiation_BS <- dta_clone_death_imp1_reinitiation %>% filter(rank == 1)
    ID_BS_reinitiation <- ID_BS_discontinuation + length(ID_BS_discontinuation)
    dta_reinitiation_BS = MC_data_preparation(data = dta_reinitiation_BS, 
                                              ID_sample = ID_BS_reinitiation)
    ### discontinuation arm t1 outcome
    dta_discontinuation_BS_1 <- dta_discontinuation_BS %>% filter(rank == 1)
    p_outcome = predict(outcome_model_discontinuation, newdata = dta_discontinuation_BS_1, type = "response")
    set.seed(060305)
    dta_discontinuation_BS_1$outcome_death = rbinom(length(p_outcome), 1, p_outcome)
    ### reinitiation arm t1 outcome
    dta_reinitiation_BS_1 <- dta_reinitiation_BS %>% filter(rank == 1)
    p_outcome = predict(outcome_model_reinitiation, newdata = dta_reinitiation_BS_1, type = "response")
    set.seed(060305)
    dta_reinitiation_BS_1$outcome_death = rbinom(length(p_outcome), 1, p_outcome)
    ### discontinuation arm t2-60 all variables
    for (i in 2 : 60) {
        dta_discontinuation_BS_per <- BS_data_generation_per(i = i, type = 'discontinuation')
        assign(paste('dta_discontinuation_BS_', i, sep = ''), 
                dta_discontinuation_BS_per)
    }
    dta_discontinuation_BS <- foreach (i = 1 : 60, .combine = rbind) %do% {
        data <- get(paste('dta_discontinuation_BS_', i, sep = ''))
    }
    ### =======================================================================
    ### reinitiation arm t2-60 all variables
    for (i in 2 : 60) {
        data = get(paste('dta_reinitiation_BS_', i - 1, sep = ''))
        dta_reinitiation_BS_per <- BS_data_generation_per(i = i, type = 'reinitiation')
        assign(paste('dta_reinitiation_BS_', i, sep = ''), dta_reinitiation_BS_per)
    }
    dta_reinitiation_BS <- foreach (i = 1 : 60, .combine = rbind) %do% {
        data <- get(paste('dta_reinitiation_BS_', i, sep = ''))
    }
    ### =======================================================================
    dta_BS <- rbind(dta_discontinuation_BS, dta_reinitiation_BS) %>% mutate(X = ifelse(X == 'reinitiation', 0, 1))
    causal_effect_estimation(dta_BS)
}
stopCluster(cl)

Warning message in e$fun(obj, substitute(ex), parent.frame(), e$data):
“already exporting variable(s): BS_data_generation_per, causal_effect_estimation, data, dta_clone_death_imp1_discontinuation, dta_clone_death_imp1_reinitiation, dta_discontinuation_MC, dta_MC, i, ID_random_sample_discontinuation, MC_data_preparation, outcome_model_discontinuation, outcome_model_reinitiation, p_outcome, x_adjusted_var, xvar_time_varying”


In [40]:
CI_point_estimation <- CI_res
CI_point_estimation <- CI_point_estimation %>% data.frame()
colnames(CI_point_estimation) <- c('risk_dis', 'risk_re', 'RD', 'RR', 'HR')
CI_point_estimation <- CI_point_estimation %>% 
    summarise(lci_risk_dis = quantile(risk_dis, 0.025), 
              uci_risk_dis = quantile(risk_dis, 0.975), 
              lci_risk_re = quantile(risk_re, 0.025), 
              uci_risk_re = quantile(risk_re, 0.975), 
              lci_RD = quantile(RD, 0.025), 
              uci_RD = quantile(RD, 0.975), 
              lci_RR = quantile(RR, 0.025), 
              uci_RR = quantile(RR, 0.975), 
              lci_HR = quantile(HR, 0.025), 
              uci_HR = quantile(HR, 0.975))
CI_point_estimation <- round(matrix(as.numeric(CI_point_estimation), nrow = 5, ncol = 2, byrow = T), 2)
causal_effect_point_estimation <- paste(t(main_point_estimation), 
                                        ' (', CI_point_estimation[ , 1], '-', 
                                        CI_point_estimation[ , 2], ')', sep = '')
names(causal_effect_point_estimation) <- colnames(main_point_estimation)
t(causal_effect_point_estimation)

3 year risk for discontinuation (%),3 year risk for reinitiation (%),risk difference (%),risk ratio,hazard ratio
62.25 (61.49-63.74),56.8 (55.28-57.49),5.44 (5.06-7.22),1.1 (1.09-1.13),1.19 (1.18-1.24)
